# Rota Planner

**Author** Alan Meeson

**Date:** 2023-10-14

This notebook gives an example of using the rota_planner package to automatically plan a rota for a set of doctors who have specified some preferences as to time off.

In [ ]:
import sys
from pyprojroot import here
sys.path.insert(0, str(here()))

In [ ]:
from typing import TypedDict, List, Dict
from enum import Enum
from itertools import product
from copy import deepcopy
from datetime import datetime, timedelta, date
from collections import defaultdict
import numpy as np
import random
import pandas as pd

In [ ]:
from rota_planner.problem import Problem
from rota_planner.shift import Shift, ShiftType
from rota_planner.doctor import Doctor, Preference
from rota_planner.template import TemplateRota, Weekday

## Constraints

### time constraints
- [x] min 11 hours between shifts
- [ ] max 7 consecutive shifts; then 48 hours off
- [x] max 72 hours in 168 hour period
- [x] max 48 hours/week average over 8 weeks
- [ ] min 40 hours/week average over 8 weeks
- [ ] max 1 in 3 weekends
- [ ] max 4 consecutive night shifts (then 46 hours off)
- [ ] max 4 consecutive on call
    - [ ] if all 4 then 48 hours rest.
     
### conditionals constraints

- once on, off for 11 hours
- once on for x, off for 46/48 hours


Assign, then merge?

Moves:
- Assign doctor to shift, if viable
- once assigned, apply any merge moves
- once assigned, apply any new constraints.
- if not viable, fail.

## Lets try it out

### Start by declaring a template and generating shifts from it

##### We'll temporarily comment out this one, as it's a little too big for the algo currently

In [ ]:
template_rota = TemplateRota()

for weekday in Weekday:
    # 2 Long day doctors 9am to 9pm (any level) 7 days a week
    template_rota.add_shift(
        day = weekday.value,
        shift_type = ShiftType.ONCALL,
        start_time = timedelta(hours=9),
        end_time = timedelta(hours=21),
        num_required = 2
    )

    # 2 doctors on nights 9pm to 9am (not F1) 7 days a week
    template_rota.add_shift(
        day = weekday.value,
        shift_type = ShiftType.NIGHT,
        start_time = timedelta(hours=21),
        end_time = timedelta(days=1, hours=9),
        num_required = 2
    )

mon_to_fri = set(Weekday) - {Weekday.SATURDAY, Weekday.SUNDAY}
for weekday in mon_to_fri:
    # 1 twilight shift 2pm to 11pm  (not F1 - monday to friday only)
    template_rota.add_shift(
        day = weekday.value,
        shift_type = ShiftType.STANDARD,
        start_time = timedelta(hours=14),
        end_time = timedelta(hours=23),
        num_required = 1
    )

    # minimum 6 doctors on standard days (9-5, m-f)  (any level) - Monday to Friday
    template_rota.add_shift(
        day = weekday.value,
        shift_type = ShiftType.STANDARD,
        start_time = timedelta(hours=9),
        end_time = timedelta(hours=17),
        num_required = 6
    )

# 1 Weekend day shift 9pm to 5pm (F1 only)
for weekday in {Weekday.SATURDAY, Weekday.SUNDAY}:
    template_rota.add_shift(
        day = weekday.value,
        shift_type = ShiftType.STANDARD,
        start_time = timedelta(hours=9),
        end_time = timedelta(hours=17),
        num_required = 1
    )



In [ ]:
# Creating a rota for 8 weeks from 1st november
shifts = template_rota.create_shifts(
    start_date=datetime(2023,11, 1), 
    num_days=14*7  # 14 weeks of rota to go from nov 1st to feb 1st
)

#### A smaller problem that the algo can currently solve

In [ ]:
template_rota = TemplateRota()

for weekday in Weekday:
    # 2 Long day doctors 9am to 9pm (any level) 7 days a week
    template_rota.add_shift(
        day = weekday.value,
        shift_type = ShiftType.ONCALL,
        start_time = timedelta(hours=9),
        end_time = timedelta(hours=21),
        num_required = 1
    )

    # 2 doctors on nights 9pm to 9am (not F1) 7 days a week
    template_rota.add_shift(
        day = weekday.value,
        shift_type = ShiftType.NIGHT,
        start_time = timedelta(hours=21),
        end_time = timedelta(days=1, hours=9),
        num_required = 1
    )

mon_to_fri = set(Weekday) - {Weekday.SATURDAY, Weekday.SUNDAY}
for weekday in mon_to_fri:
    # 1 twilight shift 2pm to 11pm  (not F1 - monday to friday only)
    template_rota.add_shift(
        day = weekday.value,
        shift_type = ShiftType.STANDARD,
        start_time = timedelta(hours=14),
        end_time = timedelta(hours=23),
        num_required = 1
    )

    # minimum 6 doctors on standard days (9-5, m-f)  (any level) - Monday to Friday
    template_rota.add_shift(
        day = weekday.value,
        shift_type = ShiftType.STANDARD,
        start_time = timedelta(hours=9),
        end_time = timedelta(hours=17),
        num_required = 1
    )

# 1 Weekend day shift 9pm to 5pm (F1 only)
for weekday in {Weekday.SATURDAY, Weekday.SUNDAY}:
    template_rota.add_shift(
        day = weekday.value,
        shift_type = ShiftType.STANDARD,
        start_time = timedelta(hours=9),
        end_time = timedelta(hours=17),
        num_required = 1
    )

In [ ]:
# Lets start with just 1 week to keep it simple
shifts = template_rota.create_shifts(
    start_date=datetime(2023,12, 25), 
    num_days=7*1
)
len(shifts)

In [ ]:
[shift for shift in shifts if shift.is_weekend_shift()]

### Now Declare some Doctors and add some days off

In [ ]:
# 20 Doctors  (5 F1s, 15 other SHOs)
num_doctors = 7
doctors = [Doctor(f"Doctor {idx}") for idx in range(num_doctors)]
len(doctors)

In [ ]:
# Everyone wants xmas day off
for doctor in doctors:
    doctor.add_preference(datetime(2023,12,25))

# Some want valentines day off
for doctor in random.sample(doctors, 2):
    doctor.add_preference(datetime(2024, 2, 14))

# Some want new years off
for doctor in random.sample(doctors, 2):
    doctor.add_preference(datetime(2024, 1, 1))

### Now lets try to solve it

Note: this will take ages to run at present because we don't have enough constraints and heuristics to reduce the search space to a small enough set.

Currently considering:
- min 11 hours off between shifts
- max 72 hours in any 168
- max 48 hours/week average over 8 weeks.
- prefer minimal clashes with prefered leave

To reduce scope, would need to look at adding: 
- Only assign weekends together (ie: don't split sat/sun between two doctors)
- Fair distribution of weekend/night shifts, ie: number per doctor roughly equal
- roughly even number of hours per doctor in total over schedule
- Prefer continuity, ie: same type of shift in a block


In [ ]:
problem = Problem(shifts, doctors)

In [ ]:
days_in_rota = list({shift.date for shift in problem.shifts})
days_in_rota.sort()

clashes = dict()
for day in days_in_rota:
    todays_shifts = [shift for shift in problem.shifts if shift.date == day]
    todays_clashes = [doctor.is_clash(shift) for doctor in problem.doctors for shift in todays_shifts]
    clashes[day] = len(todays_clashes)

# How many days to we have to disapoint everyone on
num_bad_days = len([v for v in clashes.values() if v == len(problem.doctors)])

# How bad is this dissapointment to each of the doctors
badness_score = sum([num_bad_days / len(doctor.preferences) for doctor in problem.doctors]) / len(problem.doctors)
badness_score

In [ ]:
day = days_in_rota[0]
todays_shifts = [shift for shift in problem.shifts if shift.date == day]
todays_shifts[0].date

In [ ]:
len(doctors)

In [ ]:
problem.shifts[0].date

In [ ]:
problem.calc_minimum_disapointment()
problem._min_dissapointment = 4/3  # manual hack for now.

In [ ]:
t_start = datetime.now()
solution = problem.solve()
t_end = datetime.now()
(t_end - t_start).seconds

In [ ]:
solution = problem.current_best

In [ ]:
doctors_shifts = problem.get_doctors_rota(solution, 4)
for shift in doctors_shifts:
    print(shift)

In [ ]:
num_hours = sum([shift.duration() for shift in doctors_shifts])
num_hours

In [ ]:
from collections import Counter
Counter(solution.assignments.values())

In [ ]:
all_dates = list({shift.start_time.date() for shift in shifts})
all_dates.sort()

In [ ]:
all_doctors = [doctor.name for doctor in doctors]

In [ ]:
schedule = pd.DataFrame(
    columns = all_dates,
    index = all_doctors
)
for shift_id, doctor_id in solution.assignments.items():
    shift = shifts[shift_id]
    doctor = doctors[doctor_id]
    schedule.loc[doctor.name,shift.start_time.date()] = shift.type.name

schedule[schedule.isnull()] = ShiftType.ZERO.name
schedule